In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [12]:
dataset_dir = 'plantvillage dataset'

In [13]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% train, 20% validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [14]:
# Training generator (subset='training')
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training',  # Set as training data
    shuffle=True  # Shuffle training data
)

Found 25541 images belonging to 10 classes.


In [15]:
# Validation generator (subset='validation')
val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation',  
    shuffle=True  
)

Found 6381 images belonging to 10 classes.


In [16]:
# Define ModelCheckpoint callback to save the best model
checkpoint = ModelCheckpoint(
    filepath='best_model.h5',  
    monitor='val_loss',        # Metric to monitor
    save_best_only=True,       # Save only the model with the best performance
    save_weights_only=True,    # Save only the weights, not the entire model
    verbose=1
)

In [17]:
base_model = InceptionV3(
    weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [18]:
# Unfreeze some layers in the base model for fine-tuning
base_model.trainable = True

# Freeze the first N layers (e.g., the first 100 layers)
for layer in base_model.layers[:100]:
    layer.trainable = False

In [19]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[checkpoint]
)

Epoch 1/10
400/400 [==============================] - ETA: 0s - loss: 0.2130 - accuracy: 0.9421
Epoch 1: val_loss improved from inf to 0.26082, saving model to best_model.h5
400/400 [==============================] - 669s 2s/step - loss: 0.2130 - accuracy: 0.9421 - val_loss: 0.2608 - val_accuracy: 0.9467
Epoch 2/10
400/400 [==============================] - ETA: 0s - loss: 0.0863 - accuracy: 0.9770
Epoch 2: val_loss did not improve from 0.26082
400/400 [==============================] - 487s 1s/step - loss: 0.0863 - accuracy: 0.9770 - val_loss: 0.4515 - val_accuracy: 0.8930
Epoch 3/10
400/400 [==============================] - ETA: 0s - loss: 0.0885 - accuracy: 0.9783
Epoch 3: val_loss improved from 0.26082 to 0.16417, saving model to best_model.h5
400/400 [==============================] - 465s 1s/step - loss: 0.0885 - accuracy: 0.9783 - val_loss: 0.1642 - val_accuracy: 0.9563
Epoch 4/10
400/400 [==============================] - ETA: 0s - loss: 0.0440 - accuracy: 0.9884
Epoch 4: val_

In [22]:
model.load_weights('best_model.h5')

In [23]:
model.save('complete_model.h5')